In [1]:
%%capture
!pip install numpy
!pip install librosa
!pip install matplotlib
!pip install scikit-image
!pip install opencv-python


In [2]:
# adding src to the system path
import sys
sys.path.insert(0, './src')

import os
import cv2
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from matplotlib import pyplot as plt

from dataset_loader import DataSetsLoader
from image_datasets_loader import ImageDatasetsLoader

NUM_CLASSES = 10
BATCH_SIZE = 32

# Use hardware accelerator for training
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("GPUs Available: ", physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)




2024-04-11 23:05:55.654130: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 23:05:55.674918: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 23:05:55.674936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 23:05:55.675472: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 23:05:55.679438: I tensorflow/core/platform/cpu_feature_guar

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-04-11 23:05:56.788033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-11 23:05:56.814158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-11 23:05:56.817784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
datasets_loader = DataSetsLoader()
train_points, test_points, train_labels, test_labels, CLASS_MAP = datasets_loader.transform_to_tensorflow_dataset()

image_datasets_loader = ImageDatasetsLoader()
train_points, test_points, train_labels, test_labels, CLASS_MAP_1 = image_datasets_loader.transform_to_tensorflow_dataset()

print(CLASS_MAP_1)

Using 24 number of CPU to process training data
Using 24 number of CPU to process training data
processing class: ashdro1processing class: ashpri1processing class: ashwoo2
processing class: asbfly


processed class: ashwoo2
processed class: ashpri1
processed class: ashdro1
processed class: asbfly
{0: 'ashpri1', 3: 'ashwoo2', 1: 'ashdro1', 2: 'asbfly'}


In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)



2024-04-11 23:06:01.743048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-11 23:06:01.748268: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-11 23:06:01.752755: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(255, 255, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 253, 253, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 126, 126, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 62, 62, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 128)       147584    
                                                                 
 flatten_1 (Flatten)         (None, 460800)           

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20
2/8 [======>.......................] - ETA: 0s - loss: 0.8371 - accuracy: 0.5781

2024-04-11 23:23:28.993283: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:23:29.121879: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:23:29.189470: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


6/8 [=====================>........] - ETA: 0s - loss: 94.7209 - accuracy: 0.5417 

2024-04-11 23:23:29.258063: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:23:29.324853: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:23:29.390694: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:23:29.456313: W external/local_tsl/tsl/framewor

8/8 [==============================] - 1s 82ms/step - loss: 72.6856 - accuracy: 0.5378 - val_loss: 4.1063 - val_accuracy: 0.3077
Epoch 2/20
3/8 [==========>...................] - ETA: 0s - loss: 0.8095 - accuracy: 0.6042

2024-04-11 23:23:29.702739: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 811.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


8/8 [==============================] - 1s 74ms/step - loss: 0.8024 - accuracy: 0.6135 - val_loss: 2.4071 - val_accuracy: 0.3365
Epoch 3/20
8/8 [==============================] - 1s 74ms/step - loss: 0.7886 - accuracy: 0.6135 - val_loss: 2.1436 - val_accuracy: 0.3269
Epoch 4/20
8/8 [==============================] - 1s 73ms/step - loss: 0.8036 - accuracy: 0.6135 - val_loss: 2.4884 - val_accuracy: 0.3365
Epoch 5/20
8/8 [==============================] - 1s 74ms/step - loss: 0.7763 - accuracy: 0.6135 - val_loss: 2.1141 - val_accuracy: 0.3365
Epoch 6/20
8/8 [==============================] - 1s 76ms/step - loss: 0.7702 - accuracy: 0.6135 - val_loss: 2.5043 - val_accuracy: 0.3365
Epoch 7/20
8/8 [==============================] - 1s 74ms/step - loss: 0.7611 - accuracy: 0.6096 - val_loss: 2.8405 - val_accuracy: 0.3462
Epoch 8/20
8/8 [==============================] - 1s 74ms/step - loss: 0.7656 - accuracy: 0.6096 - val_loss: 2.5603 - val_accuracy: 0.3269
Epoch 9/20
8/8 [======================

In [14]:
current_folder = os.getcwd()
path = os.path.join(current_folder, "data/train_data/asbfly/XC134896.png")
img = cv2.imread(path, cv2.IMREAD_COLOR)
print(img.shape)

prediction = model.predict(numpy.expand_dims(img, axis=0))
print(prediction)
print(CLASS_MAP)


(255, 255, 3)
1/1 [==============================] - 0s 125ms/step
[[ 0.08708845  0.14235128  0.05126365 -0.26109913]]
{0: 'ashpri1', 1: 'ashdro1', 2: 'asbfly', 3: 'ashwoo2'}
